# Fiona

- Docs: http://toblerity.org/fiona/fiona.html
- More recent: http://juanlu001-fiona.readthedocs.io/en/latest/

In [30]:
import os
import zipfile
import geopandas as gpd

In [3]:
datadir = os.path.join("/media", "disk", "TIGER Data")
gpd.GeoDataFrame.from_file(os.path.join(datadir, 'tl_2016_17031_roads')).head()

,FULLNAME,LINEARID,MTFCC,RTTYP,geometry
0,47th Pl Exd,110380277026,S1400,M,"LINESTRING (-87.595765 41.81460799999999, -87...."
1,Golden Spr,110380298305,S1400,M,"LINESTRING (-87.921616 41.67378399999999, -87...."
2,Edens Expy Spr,1104259027148,S1100,M,LINESTRING (-87.86542699999998 42.149693999999...
3,Edens Expy Spr,1104259564382,S1100,M,"LINESTRING (-87.832262 42.14520399999999, -87...."
4,Edens Expy Spr,1104472109755,S1100,M,"LINESTRING (-87.874499 42.152788, -87.87439399..."


In [5]:
import fiona

In [16]:
with fiona.open(os.path.join(datadir, 'tl_2016_17031_roads')) as inp:
    print(inp.schema)
    print(inp.crs)
    data = list(inp)

{'properties': OrderedDict([('LINEARID', 'str:22'), ('FULLNAME', 'str:100'), ('RTTYP', 'str:1'), ('MTFCC', 'str:5')]), 'geometry': 'LineString'}
{'init': 'epsg:4269'}


In [17]:
data[0]

{'geometry': {'coordinates': [(-87.595765, 41.814607999999986),
   (-87.595702, 41.81463399999999),
   (-87.59551599999999, 41.81471299999999),
   (-87.59545399999999, 41.81474)],
  'type': 'LineString'},
 'id': '0',
 'properties': OrderedDict([('LINEARID', '110380277026'),
              ('FULLNAME', '47th Pl Exd'),
              ('RTTYP', 'M'),
              ('MTFCC', 'S1400')]),
 'type': 'Feature'}

In [34]:
filename = os.path.join(datadir, 'tl_2016_17031_roads.zip')

zf = zipfile.ZipFile(filename)
zf.filelist

[<ZipInfo filename='tl_2016_17031_roads.cpg' filemode='-rwxrwxr-x' file_size=5>,
 <ZipInfo filename='tl_2016_17031_roads.dbf' compress_type=deflate filemode='-rwxrwxr-x' file_size=10236312 compress_size=771563>,
 <ZipInfo filename='tl_2016_17031_roads.prj' compress_type=deflate filemode='-rwxrwxr-x' file_size=165 compress_size=133>,
 <ZipInfo filename='tl_2016_17031_roads.shp' compress_type=deflate filemode='-rwxrwxr-x' file_size=23335716 compress_size=13720745>,
 <ZipInfo filename='tl_2016_17031_roads.shp.ea.iso.xml' compress_type=deflate filemode='-rwxrwxrwx' file_size=20621 compress_size=2000>,
 <ZipInfo filename='tl_2016_17031_roads.shp.iso.xml' compress_type=deflate filemode='-rwxrwxrwx' file_size=291967 compress_size=11976>,
 <ZipInfo filename='tl_2016_17031_roads.shp.xml' compress_type=deflate filemode='-rwxrwxrwx' file_size=92101 compress_size=8070>,
 <ZipInfo filename='tl_2016_17031_roads.shx' compress_type=deflate filemode='-rwxrwxr-x' file_size=634900 compress_size=308395>]

In [45]:
fiona.listlayers(".", vfs="zip://"+filename)

ValueError: No data available at path '/vsizip//media/disk/TIGER Data/tl_2016_17031_roads.zip.'

In [42]:
with fiona.open("./tl_2016_17031_roads", vfs="zip://"+filename) as inp:
    pass

FionaValueError: No dataset found at path '/vsizip//media/disk/TIGER Data/tl_2016_17031_roads.zip./tl_2016_17031_roads' using drivers: *

In [40]:
fiona.__version__

'1.7.3'